In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

import libs.split_data as sd

In [701]:
train_set = pd.read_csv('data/train/matchups_train.csv')
test_set = pd.read_csv('data/test/matchups_test.csv')

In [702]:
mp = sd.MatchupPrep()

train_set = mp.game_date_to_index(train_set)
test_set = mp.game_date_to_index(test_set)

In [703]:
columns_used = [col for col in train_set.columns if ('cluster' in col)]
columns_used.extend(['batter', 'pitcher', 'bb', 'events', 'pa', 'estimated_ba_using_speedangle', 'estimated_ba_using_speedangle_list'])

In [704]:
train_set = train_set[columns_used].copy()
test_set = test_set[columns_used].copy()

# Pitchers

In [399]:
def rolling_cluster_hard(data,depth_num,depth_type='D'):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster' in col and 'attribute' not in col and 'list' not in col]
    cluster_cols.extend(['pa','bb'])
    col_names = [col + '_roll' for col in cluster_cols]
    depth = str(depth_num) + depth_type
    
    data.index = pd.to_datetime(data.index)
    
    temp_data = pd.DataFrame()
    temp_data = data.groupby(['pitcher'], as_index=False).rolling(depth)[cluster_cols].sum()
    temp_data.columns = col_names
    temp_data.reset_index(level=0)
    
    temp_data['pitcher_pa'] = temp_data.pa_roll + temp_data.bb_roll
    
    data = data.merge(temp_data, on=['game_date','pitcher'])
    
    return correct_hard_roll(data)
    

def correct_hard_roll(data):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster' in col and 'attribute' not in col and 'list' not in col and 'roll' not in col]
    cluster_cols.extend(['pa','bb'])
    col_names = [col + '_roll' for col in cluster_cols]
    
    for i in range(len(cluster_cols)):
        roll_item = col_names[i]
        item = cluster_cols[i]
        
        data[roll_item] = data[roll_item] - data[item]
    
    data['pitcher_pa'] = data['pitcher_pa'] - data['pa'] - data['bb']
    return data

In [400]:
def hard_per_pa(data):
    cluster_cols = [col for col in data.columns if 'cluster' in col and 'attribute' not in col and 'list' not in col and 'roll' in col]
    col_names = [item + '_per_pa' for item in cluster_cols]
    
    for i in range(len(col_names)):
        data[col_names[i]] = data[cluster_cols[i]] / data['pitcher_pa']
        
    return data

In [401]:
temp_df = train_set.copy()
#temp_df.groupby(['pitcher',pd.Grouper(freq='20D')])[[col for col in temp_df if 'cluster' in col and 'attribute' not in col and 'list' not in col]].sum()
temp_df = rolling_cluster_hard(temp_df,20)
temp_df = hard_per_pa(temp_df)

In [402]:
temp_df

,cluster_attribute_0,cluster_attribute_0_max,cluster_attribute_0_min,cluster_attribute_0_list,cluster_attribute_1,cluster_attribute_1_max,cluster_attribute_1_min,cluster_attribute_1_list,cluster_attribute_2,cluster_attribute_2_max,cluster_attribute_2_min,cluster_attribute_2_list,cluster_attribute_3,cluster_attribute_3_max,cluster_attribute_3_min,cluster_attribute_3_list,cluster_attribute_4,cluster_attribute_4_max,cluster_attribute_4_min,cluster_attribute_4_list,cluster_attribute_5,cluster_attribute_5_max,cluster_attribute_5_min,cluster_attribute_5_list,cluster_attribute_6,cluster_attribute_6_max,cluster_attribute_6_min,cluster_attribute_6_list,cluster_attribute_7,cluster_attribute_7_max,cluster_attribute_7_min,cluster_attribute_7_list,cluster_attribute_8,cluster_attribute_8_max,cluster_attribute_8_min,cluster_attribute_8_list,cluster_attribute_9,cluster_attribute_9_max,cluster_attribute_9_min,cluster_attribute_9_list,cluster_attribute_10,cluster_attribute_10_max,cluster_attribute_10_min,cluster_attribute_10_list,cluster_0,cluster_0_list,cluster_1,cluster_1_list,cluster_2,cluster_2_list,cluster_3,cluster_3_list,cluster_4,cluster_4_list,cluster_5,cluster_5_list,cluster_6,cluster_6_list,cluster_7,cluster_7_list,cluster_8,cluster_8_list,cluster_9,cluster_9_list,cluster_10,cluster_10_list,batter,pitcher,bb,events,pa,estimated_ba_using_speedangle,estimated_ba_using_speedangle_list,cluster_0_roll,cluster_1_roll,cluster_2_roll,cluster_3_roll,cluster_4_roll,cluster_5_roll,cluster_6_roll,cluster_7_roll,cluster_8_roll,cluster_9_roll,cluster_10_roll,pa_roll,bb_roll,pitcher_pa,cluster_0_roll_per_pa,cluster_1_roll_per_pa,cluster_2_roll_per_pa,cluster_3_roll_per_pa,cluster_4_roll_per_pa,cluster_5_roll_per_pa,cluster_6_roll_per_pa,cluster_7_roll_per_pa,cluster_8_roll_per_pa,cluster_9_roll_per_pa,cluster_10_roll_per_pa
game_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-02,0.108550,0.108550,0.108550,[0.1085496446624735],0.122777,0.122777,0.122777,[0.1227769918664719],0.082354,0.082354,0.082354,[0.0823542335673734],0.065391,0.065391,0.065391,[0.065391023980825],0.082577,0.082577,0.082577,[0.082576740305458],0.036663,0.036663,0.036663,[0.0366625523871585],0.122960,0.122960,0.122960,[0.122959651422464],0.082423,0.082423,0.082423,[0.0824234461333729],0.135249,0.135249,0.135249,[0.135248692978269],0.036661,0.036661,0.036661,[0.036660554441494],0.124396,0.124396,0.124396,[0.1243964682546394],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],1.0,[1],0.0,[0],0.0,[0],425844.0,518516.0,0.0,['field_out'],1,0.4730,[0.473],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-02,0.082661,0.083282,0.082039,"[0.0820389727339067, 0.0832823314173698]",0.142560,0.149517,0.135603,"[0.1495174503842276, 0.1356033501504556]",0.073797,0.076366,0.071227,"[0.0712270840402173, 0.0763661083408702]",0.064850,0.066858,0.062842,"[0.0628415495337317, 0.0668575506523015]",0.073930,0.076505,0.071354,"[0.0713541793293925, 0.0765051836891]",0.041905,0.042997,0.040813,"[0.0408125681411728, 0.0429970596047767]",0.102914,0.106763,0.099064,"[0.099064209408037, 0.1067630423077841]",0.073838,0.076409,0.071267,"[0.0712665206357332, 0.0764092806449629]",0.156912,0.159762,0.154063,"[0.1597616278102806, 0.154062725508934]",0.041903,0.042995,0.040811,"[0.0408107952221454, 0.0429951782456229]",0.144732,0.151305,0.138158,"[0.1513050427611547, 0.1381581894378217]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",2.0,"[1, 1]",0.0,"[0, 0]",0.0,"[0, 0]",452657.0,593372.0,0.0,"['field_out', 'strikeout']",2,0.0665,"[0.133, 0.0]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-02,0.041997,0.043197,0.040796,"[0.0431971962190918, 0.0407964334848539]",0.155361,0.272603,0.038118,"[0.0381182491625044, 0.2726028627524947]",0.065036,0.097922,0.032150,"[0.0979222

In [403]:
from sklearn.ensemble import RandomForestRegressor

In [412]:
train = train_set.copy()
temp_train = rolling_cluster_hard(train,20)
temp_train = hard_per_pa(temp_train)

cols_for_use = [col for col in temp_train.columns if ('attribute' not in col) & ('list' not in col) & ('events' not in col) & ('cluster' not in col)]
cols_to_add = [col for col in temp_train.columns if ('cluster' in col) & ('roll' in col)]
cols_for_use.extend(cols_to_add)
cols_for_use.remove('batter')
cols_for_use.remove('pitcher')
cols_for_use.remove('bb')
cols_for_use.remove('pa')

temp_train = temp_train[cols_for_use]
temp_train.dropna(inplace=True)

In [413]:
test = test_set.copy()
temp_test = rolling_cluster_hard(test,20)
temp_test = hard_per_pa(temp_test)

In [414]:
rf = RandomForestRegressor()
rf.fit(temp_train.loc[:,temp_train.columns!='estimated_ba_using_speedangle'],temp_train.loc[:,temp_train.columns=='estimated_ba_using_speedangle'])

D:\Python\Anaconda3_2020.11\envs\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor()

In [415]:
cols_for_use.remove('estimated_ba_using_speedangle')

feature_importance = {}

for i in range(len(temp_train[cols_for_use].columns)):
    feature_importance[temp_train[cols_for_use].columns[i]] = rf.feature_importances_[i]

In [416]:
{k: v for k, v in sorted(feature_importance.items(), key=lambda item: item[1],reverse=True)}

{'cluster_8_roll_per_pa': 0.0872279487099619,
 'pa_roll': 0.08031399809721627,
 'cluster_1_roll_per_pa': 0.07906567277252818,
 'cluster_10_roll_per_pa': 0.06207077022985825,
 'cluster_3_roll_per_pa': 0.06150239780994781,
 'cluster_6_roll_per_pa': 0.06103808931687689,
 'bb_roll': 0.05855366538490889,
 'cluster_9_roll_per_pa': 0.0573890303821553,
 'cluster_1_roll': 0.04946105960190941,
 'cluster_8_roll': 0.04708551203516667,
 'cluster_5_roll_per_pa': 0.04377721080636585,
 'pitcher_pa': 0.043403262142071186,
 'cluster_6_roll': 0.03577009226705042,
 'cluster_3_roll': 0.0317544487248679,
 'cluster_0_roll_per_pa': 0.031306177304500686,
 'cluster_4_roll_per_pa': 0.029253071487111836,
 'cluster_10_roll': 0.028946924532023112,
 'cluster_9_roll': 0.027377297929600836,
 'cluster_0_roll': 0.024560993297428745,
 'cluster_5_roll': 0.017954317803535274,
 'cluster_2_roll_per_pa': 0.017289613110777334,
 'cluster_4_roll': 0.014236733308989936,
 'cluster_2_roll': 0.00801409810243211,
 'cluster_7_roll': 0

In [417]:
def rolling_cluster_soft(data,depth_num,depth_type="D"):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster_attribute' in col and 'list' not in col]
    col_names = [col + '_roll' for col in cluster_cols]
    depth = str(depth_num) + depth_type
    
    data.index = pd.to_datetime(data.index)
    
    temp_data = pd.DataFrame()
    temp_data = data.groupby(['pitcher'], as_index=False).rolling(depth)[cluster_cols].mean()
    temp_data.columns = col_names
    temp_data.reset_index(level=0)
    
    data = data.merge(temp_data, on=['game_date','pitcher'])
    
    return correct_soft_roll(data)

def correct_soft_roll(data):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster' in col and 'attribute' in col and 'list' not in col and 'roll' not in col]
    cluster_cols.extend(['pa','bb'])
    col_names = [col + '_roll' for col in cluster_cols]
    
    data['pitcher_pa'] = data['pitcher_pa'] - data['pa'] - data['bb']
    
    
    for i in range(len(cluster_cols)):
        roll_item = col_names[i]
        item = cluster_cols[i]
        
        data[roll_item] = (data[roll_item] * (data['pitcher_pa'] + data['pa'] + data['bb']) - data[item]) / data['pitcher_pa']
    
    return data

In [419]:
temp_df = train_set.copy()
#temp_df.groupby(['pitcher',pd.Grouper(freq='20D')])[[col for col in temp_df if 'cluster' in col and 'attribute' not in col and 'list' not in col]].sum()
temp_df = rolling_cluster_soft(temp_df,20)

In [425]:
train = train_set.copy()
temp_train = rolling_cluster_soft(train,20)

cols_for_use = [col for col in temp_train.columns if ('attribute' not in col) & ('list' not in col) & ('events' not in col) & ('cluster' not in col)]
cols_to_add = [col for col in temp_train.columns if ('cluster' in col) & ('roll' in col)]
cols_for_use.extend(cols_to_add)
cols_for_use.remove('batter')
cols_for_use.remove('pitcher')
cols_for_use.remove('bb')
cols_for_use.remove('pa')

temp_train = temp_train[cols_for_use]
temp_train = temp_train.loc[temp_train.pitcher_pa>0,:]
temp_train.dropna(inplace=True)

In [426]:
rf = RandomForestRegressor()
rf.fit(temp_train.loc[:,temp_train.columns!='estimated_ba_using_speedangle'],temp_train.loc[:,temp_train.columns=='estimated_ba_using_speedangle'])

D:\Python\Anaconda3_2020.11\envs\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor()

In [427]:
cols_for_use.remove('estimated_ba_using_speedangle')

feature_importance = {}

for i in range(len(temp_train[cols_for_use].columns)):
    feature_importance[temp_train[cols_for_use].columns[i]] = rf.feature_importances_[i]

In [428]:
{k: v for k, v in sorted(feature_importance.items(), key=lambda item: item[1],reverse=True)}

{'cluster_attribute_0_min_roll': 0.06185861873917519,
 'cluster_attribute_8_max_roll': 0.054302497951249584,
 'cluster_attribute_0_max_roll': 0.05297497500283728,
 'cluster_attribute_0_roll': 0.04939697057316327,
 'cluster_attribute_6_min_roll': 0.04930340469438013,
 'cluster_attribute_8_min_roll': 0.04878420298964211,
 'cluster_attribute_6_max_roll': 0.04775315264314228,
 'cluster_attribute_8_roll': 0.042041516222114,
 'cluster_attribute_6_roll': 0.04039001508395137,
 'cluster_attribute_1_max_roll': 0.02875904329767741,
 'cluster_attribute_3_min_roll': 0.027315857561720964,
 'cluster_attribute_5_min_roll': 0.02719119679625729,
 'cluster_attribute_1_roll': 0.026925334466047644,
 'cluster_attribute_1_min_roll': 0.026589907768992886,
 'cluster_attribute_9_min_roll': 0.02631036125034987,
 'cluster_attribute_3_roll': 0.025266766253317842,
 'cluster_attribute_3_max_roll': 0.025107813479043117,
 'cluster_attribute_10_max_roll': 0.024687507923165625,
 'pa_roll': 0.024579521380850823,
 'cluste

# Batters

In [519]:
from ast import literal_eval

In [573]:
def explode(data):
    data = data.copy()
    
    keep_cols = [col for col in data.columns if 'list' in col]
    keep_cols.extend(['batter'])
    
    data = data[keep_cols]
    keep_cols.remove('batter')
        
    for col in keep_cols:
        data[col] = data[col].apply(lambda x: x.translate({ord(i): None for i in ' []'}).split(','))
    
    data = data.set_index(['batter',data.index]).apply(pd.Series.explode).reset_index(level=0)
    data = data.replace('nan',np.nan)
    data.dropna(inplace=True)
    data[keep_cols] = data[keep_cols].astype('float')
    return data

In [648]:
def cluster_day(data):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster' in col]
    col_names = [col + '_agg' for col in cluster_cols]
    
    data[col_names] = data.loc[:,cluster_cols].multiply(data.loc[:,'estimated_ba_using_speedangle_list'], axis="index")
    
    agg_dict = {}
    for col in cluster_cols:
        agg_dict[col] = 'sum'
    for col in col_names:
        agg_dict[col] = 'mean'
        
    data = data.groupby(['batter','game_date']).agg(agg_dict).reset_index(level=0)
    data[col_names] = data[col_names].replace(0,np.nan)
    return data
    

In [706]:
def rolling_xba(data,depth_num,depth_type='D'):
    data = data.copy()
    cluster_cols = [col for col in data.columns if 'agg' in col]
    col_names = [col.replace('_agg','') + '_expected' for col in cluster_cols]
    
    depth = str(depth_num) + depth_type
    
    temp_data = pd.DataFrame()
    temp_data = data.groupby(['batter','game_date'], as_index=False).rolling(depth)[cluster_cols].mean()
    temp_data.columns = col_names
    temp_data.drop_duplicates(inplace=False)
    temp_data = temp_data.reset_index(level=0).reset_index(level=0)
    temp_data.drop('game_date',axis=1, inplace=True)    
    
    data = data.merge(temp_data, on=['game_date','batter'])
    
    return data

def rolling_sum_xba(data,depth_num,depth_type='D'):
    data = data.copy()
    
    cluster_cols_sum = [col for col in data.columns if 'cluster' in col and 'agg' not in col and 'expected' not in col]
    col_names_sum = [col + '_sum' for col in cluster_cols_sum]
    print(cluster_cols_sum)
    depth = str(depth_num) + depth_type
    
    temp_df = pd.DataFrame()
    temp_df = data.groupby(['batter','game_date'], as_index=False).rolling(depth)[cluster_cols_sum].sum()
    temp_df.columns = col_names_sum
    temp_df.drop_duplicates(inplace=False)
    temp_df = temp_df.reset_index(level=0).reset_index(level=0)
    temp_df.drop('game_date',axis=1, inplace=True)
    
    data = data.merge(temp_df, on=['game_date','batter'])
    
    return data

def correct_rolling_xba(data):
    expected_cols = [col for col in data.columns if '_expected' in col]
    agg_cols = [col for col in data.columns if '_agg' in col]
    cluster_cols = [col for col in data.columns if 'cluster' in col and '_agg' not in col and '_expected' not in col and '_sum' not in col]
    sum_cols = [col for col in data.columns if '_sum' in col]
    counter = 0
    
    for i in range(len(expected_cols)):
        m = data[expected_cols[i]]
        n = data[sum_cols[i]]
        x = data[agg_cols[i]]
        t = data[cluster_cols[i]]
        
        if 'attribute' in expected_cols[i]:
            name = 'cluster_attribute_' + str(i) + '_xba'
            counter += 1
        elif 'attribute' not in expected_cols[i]:
            name = 'cluster_' + str(i%counter) + '_xba'
            
        data[name] = ((m * n) - x) / (n - t)
    
    return data

In [707]:

temp_df = explode(train_set)
temp_df = cluster_day(temp_df)
temp_df = rolling_xba_hard(temp_df,20)
temp_df = rolling_sum_xba(temp_df,20)
temp_df = correct_rolling_xba(temp_df)

['cluster_attribute_0_list', 'cluster_attribute_1_list', 'cluster_attribute_2_list', 'cluster_attribute_3_list', 'cluster_attribute_4_list', 'cluster_attribute_5_list', 'cluster_attribute_6_list', 'cluster_attribute_7_list', 'cluster_attribute_8_list', 'cluster_attribute_9_list', 'cluster_attribute_10_list', 'cluster_0_list', 'cluster_1_list', 'cluster_2_list', 'cluster_3_list', 'cluster_4_list', 'cluster_5_list', 'cluster_6_list', 'cluster_7_list', 'cluster_8_list', 'cluster_9_list', 'cluster_10_list']


In [705]:
train_set[train_set.batter == 592450]

,cluster_attribute_0,cluster_attribute_0_max,cluster_attribute_0_min,cluster_attribute_0_list,cluster_attribute_1,cluster_attribute_1_max,cluster_attribute_1_min,cluster_attribute_1_list,cluster_attribute_2,cluster_attribute_2_max,cluster_attribute_2_min,cluster_attribute_2_list,cluster_attribute_3,cluster_attribute_3_max,cluster_attribute_3_min,cluster_attribute_3_list,cluster_attribute_4,cluster_attribute_4_max,cluster_attribute_4_min,cluster_attribute_4_list,cluster_attribute_5,cluster_attribute_5_max,cluster_attribute_5_min,cluster_attribute_5_list,cluster_attribute_6,cluster_attribute_6_max,cluster_attribute_6_min,cluster_attribute_6_list,cluster_attribute_7,cluster_attribute_7_max,cluster_attribute_7_min,cluster_attribute_7_list,cluster_attribute_8,cluster_attribute_8_max,cluster_attribute_8_min,cluster_attribute_8_list,cluster_attribute_9,cluster_attribute_9_max,cluster_attribute_9_min,cluster_attribute_9_list,cluster_attribute_10,cluster_attribute_10_max,cluster_attribute_10_min,cluster_attribute_10_list,cluster_0,cluster_0_list,cluster_1,cluster_1_list,cluster_2,cluster_2_list,cluster_3,cluster_3_list,cluster_4,cluster_4_list,cluster_5,cluster_5_list,cluster_6,cluster_6_list,cluster_7,cluster_7_list,cluster_8,cluster_8_list,cluster_9,cluster_9_list,cluster_10,cluster_10_list,batter,pitcher,bb,events,pa,estimated_ba_using_speedangle,estimated_ba_using_speedangle_list
game_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [708]:
train_set.sort_index()

,cluster_attribute_0,cluster_attribute_0_max,cluster_attribute_0_min,cluster_attribute_0_list,cluster_attribute_1,cluster_attribute_1_max,cluster_attribute_1_min,cluster_attribute_1_list,cluster_attribute_2,cluster_attribute_2_max,cluster_attribute_2_min,cluster_attribute_2_list,cluster_attribute_3,cluster_attribute_3_max,cluster_attribute_3_min,cluster_attribute_3_list,cluster_attribute_4,cluster_attribute_4_max,cluster_attribute_4_min,cluster_attribute_4_list,cluster_attribute_5,cluster_attribute_5_max,cluster_attribute_5_min,cluster_attribute_5_list,cluster_attribute_6,cluster_attribute_6_max,cluster_attribute_6_min,cluster_attribute_6_list,cluster_attribute_7,cluster_attribute_7_max,cluster_attribute_7_min,cluster_attribute_7_list,cluster_attribute_8,cluster_attribute_8_max,cluster_attribute_8_min,cluster_attribute_8_list,cluster_attribute_9,cluster_attribute_9_max,cluster_attribute_9_min,cluster_attribute_9_list,cluster_attribute_10,cluster_attribute_10_max,cluster_attribute_10_min,cluster_attribute_10_list,cluster_0,cluster_0_list,cluster_1,cluster_1_list,cluster_2,cluster_2_list,cluster_3,cluster_3_list,cluster_4,cluster_4_list,cluster_5,cluster_5_list,cluster_6,cluster_6_list,cluster_7,cluster_7_list,cluster_8,cluster_8_list,cluster_9,cluster_9_list,cluster_10,cluster_10_list,batter,pitcher,bb,events,pa,estimated_ba_using_speedangle,estimated_ba_using_speedangle_list
game_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-02,0.108550,0.108550,0.108550,[0.1085496446624735],0.122777,0.122777,0.122777,[0.1227769918664719],0.082354,0.082354,0.082354,[0.0823542335673734],0.065391,0.065391,0.065391,[0.065391023980825],0.082577,0.082577,0.082577,[0.082576740305458],0.036663,0.036663,0.036663,[0.0366625523871585],0.122960,0.122960,0.122960,[0.122959651422464],0.082423,0.082423,0.082423,[0.0824234461333729],0.135249,0.135249,0.135249,[0.135248692978269],0.036661,0.036661,0.036661,[0.036660554441494],0.124396,0.124396,0.124396,[0.1243964682546394],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],0.0,[0],1.0,[1],0.0,[0],0.0,[0],425844.0,518516.0,0.0,['field_out'],1,0.4730,[0.473]
2017-04-02,0.082661,0.083282,0.082039,"[0.0820389727339067, 0.0832823314173698]",0.142560,0.149517,0.135603,"[0.1495174503842276, 0.1356033501504556]",0.073797,0.076366,0.071227,"[0.0712270840402173, 0.0763661083408702]",0.064850,0.066858,0.062842,"[0.0628415495337317, 0.0668575506523015]",0.073930,0.076505,0.071354,"[0.0713541793293925, 0.0765051836891]",0.041905,0.042997,0.040813,"[0.0408125681411728, 0.0429970596047767]",0.102914,0.106763,0.099064,"[0.099064209408037, 0.1067630423077841]",0.073838,0.076409,0.071267,"[0.0712665206357332, 0.0764092806449629]",0.156912,0.159762,0.154063,"[0.1597616278102806, 0.154062725508934]",0.041903,0.042995,0.040811,"[0.0408107952221454, 0.0429951782456229]",0.144732,0.151305,0.138158,"[0.1513050427611547, 0.1381581894378217]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",2.0,"[1, 1]",0.0,"[0, 0]",0.0,"[0, 0]",452657.0,593372.0,0.0,"['field_out', 'strikeout']",2,0.0665,"[0.133, 0.0]"
2017-04-02,0.041997,0.043197,0.040796,"[0.0431971962190918, 0.0407964334848539]",0.155361,0.272603,0.038118,"[0.0381182491625044, 0.2726028627524947]",0.065036,0.097922,0.032150,"[0.0979222708847594, 0.0321498186416542]",0.069371,0.111254,0.027488,"[0.111253643555271, 0.0274881996010879]",0.064978,0.097733,0.032224,"[0.0977325362965475, 0.032223794147312]",0.098745,0.180637,0.016853,"[0.1806374416241463, 0.0168531684574633]",0.062047,0.071412,0.052681,"[0.0714124110235777, 0.0526807169381467]",0.065018,0.097863,0.032173,"[0.0978633179828019, 0.0321727620004981]",0.126037,0.209461,0.042612,"[0.0426119974563655, 0.2094613046111595]",0.098750,0.180648,0.016852,"[0.1806479815549347, 0.0168523525440075]",0.152661,0.266719,0.038603,"[0.0386029542399993, 0.2667185868213217]",0.0,"[0, 0]",1.0,"[0, 1]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[0, 0]",0.0,"[

In [692]:
train_set['batter'].unique

<bound method Series.unique of game_date
2017-04-02    425844.0
2017-04-02    452657.0
2017-04-02    518516.0
2017-04-02    593372.0
2017-04-03    592789.0
                ...   
2021-10-02    471911.0
2021-10-02    452657.0
2021-10-02    668820.0
2021-10-02    605397.0
2021-10-02    680686.0
Name: batter, Length: 10419, dtype: float64>

In [695]:
temp_df.sort_index().index

DatetimeIndex(['2017-04-02', '2017-04-02', '2017-04-02', '2017-04-02',
               '2017-04-03', '2017-04-03', '2017-04-03', '2017-04-03',
               '2017-04-03', '2017-04-03',
               ...
               '2021-10-02', '2021-10-02', '2021-10-02', '2021-10-02',
               '2021-10-02', '2021-10-02', '2021-10-02', '2021-10-02',
               '2021-10-02', '2021-10-02'],
              dtype='datetime64[ns]', name='game_date', length=9976, freq=None)

In [568]:
def rolling_cluster_hard(data,depth_num,depth_type='D'):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster' in col and 'attribute' not in col and 'list' not in col]
    cluster_cols.extend(['pa','bb'])
    col_names = [col + '_roll' for col in cluster_cols]
    depth = str(depth_num) + depth_type
    
    data.index = pd.to_datetime(data.index)
    
    temp_data = pd.DataFrame()
    temp_data = data.groupby(['pitcher'], as_index=False).rolling(depth)[cluster_cols].sum()
    temp_data.columns = col_names
    temp_data.reset_index(level=0)
    
    temp_data['pitcher_pa'] = temp_data.pa_roll + temp_data.bb_roll
    
    data = data.merge(temp_data, on=['game_date','pitcher'])
    
    return correct_hard_roll(data)
    

def correct_hard_roll(data):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster' in col and 'attribute' not in col and 'list' not in col and 'roll' not in col]
    col_names = [col + '_roll' for col in cluster_cols]
    
    for i in range(len(cluster_cols)):
        roll_item = col_names[i]
        item = cluster_cols[i]
        
        data[roll_item] = data[roll_item] - data[item]
    
    data['pitcher_pa'] = data['pitcher_pa'] - data['pa'] - data['bb']
    return data

In [549]:
temp_df

,batter,cluster_attribute_0_list,cluster_attribute_1_list,cluster_attribute_2_list,cluster_attribute_3_list,cluster_attribute_4_list,cluster_attribute_5_list,cluster_attribute_6_list,cluster_attribute_7_list,cluster_attribute_8_list,cluster_attribute_9_list,cluster_attribute_10_list,cluster_0_list,cluster_1_list,cluster_2_list,cluster_3_list,cluster_4_list,cluster_5_list,cluster_6_list,cluster_7_list,cluster_8_list,cluster_9_list,cluster_10_list,estimated_ba_using_speedangle_list
game_date,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-02,425844.0,0.1085496446624735,0.1227769918664719,0.0823542335673734,0.065391023980825,0.082576740305458,0.0366625523871585,0.122959651422464,0.0824234461333729,0.135248692978269,0.036660554441494,0.1243964682546394,0,0,0,0,0,0,0,0,1,0,0,0.473
2017-04-02,452657.0,0.0820389727339067,0.1495174503842276,0.0712270840402173,0.0628415495337317,0.0713541793293925,0.0408125681411728,0.099064209408037,0.0712665206357332,0.1597616278102806,0.0408107952221454,0.1513050427611547,0,0,0,0,0,0,0,0,1,0,0,0.133
2017-04-02,452657.0,0.0832823314173698,0.1356033501504556,0.0763661083408702,0.0668575506523015,0.0765051836891,0.0429970596047767,0.1067630423077841,0.0764092806449629,0.154062725508934,0.0429951782456229,0.1381581894378217,0,0,0,0,0,0,0,0,1,0,0,0.0
2017-04-02,518516.0,0.0431971962190918,0.0381182491625044,0.0979222708847594,0.111253643555271,0.0977325362965475,0.1806374416241463,0.0714124110235777,0.0978633179828019,0.0426119974563655,0.1806479815549347,0.0386029542399993,0,0,0,0,0,0,0,0,0,1,0,0.927
2017-04-02,593372.0,0.1968063758531448,0.0878486194009949,0.0887357067393223,0.0743974516454011,0.0889247953099637,0.0440161227263659,0.1022041251989169,0.0887945041120075,0.0954521077728203,0.0440138419496801,0.0888063492913819,1,0,0,0,0,0,0,0,0,0,0,0.127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-02,471911.0,0.2758303080079201,0.0936216821620034,0.0714733103034638,0.0612502682307298,0.0716252329413028,0.0383333193812856,0.0886188405140195,0.0715204810258638,0.0954374763723108,0.0383315432594185,0.0939575378016813,1,0,0,0,0,0,0,0,0,0,0,0.074
2021-10-02,452657.0,0.0529753941395533,0.1634588509429339,0.0671381471555999,0.0587107318386314,0.0672580420817857,0.0384498823939632,0.1080415122027056,0.0671753759426245,0.1732594066546856,0.0384481829915746,0.1650844736559417,0,0,0,0,0,0,0,0,1,0,0,0.0
2021-10-02,668820.0,0.0692008770633487,0.1629960865291326,0.0695363009818592,0.0627356991459097,0.0696427524130636,0.0424194642113251,0.0951262772845982,0.0695693177240443,0.1541930126118479,0.042417662888557,0.1621625491463132,0,1,0,0,0,0,0,0,0,0,0,0.0


In [551]:
cols_c = [col for col in temp_df.columns if 'cluster' in col and 'attribute' not in col]

temp_df[cols_c].nunique()

cluster_0_list     2
cluster_1_list     2
cluster_2_list     2
cluster_3_list     2
cluster_4_list     2
cluster_5_list     2
cluster_6_list     2
cluster_7_list     2
cluster_8_list     2
cluster_9_list     2
cluster_10_list    2
dtype: int64

In [ ]:
def rolling_cluster_soft(data,depth_num,depth_type="D"):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster_attribute' in col and 'list' not in col]
    cluster_cols.extend(['pa','bb'])
    col_names = [col + '_roll' for col in cluster_cols]
    depth = str(depth_num) + depth_type
    
    data.index = pd.to_datetime(data.index)
    
    temp_data = pd.DataFrame()
    temp_data = data.groupby(['pitcher'], as_index=False).rolling(depth)[cluster_cols].mean()
    temp_data.columns = col_names
    temp_data.reset_index(level=0)
    
    temp_data['pitcher_pa'] = temp_data.pa_roll + temp_data.bb_roll
    
    data = data.merge(temp_data, on=['game_date','pitcher'])
    
    return correct_soft_roll(data)

def correct_soft_roll(data):
    data = data.copy()
    
    cluster_cols = [col for col in data.columns if 'cluster' in col and 'attribute' in col and 'list' not in col and 'roll' not in col]
    cluster_cols.extend(['pa','bb'])
    col_names = [col + '_roll' for col in cluster_cols]
    
    data['pitcher_pa'] = data['pitcher_pa'] - data['pa'] - data['bb']
    
    
    for i in range(len(cluster_cols)):
        roll_item = col_names[i]
        item = cluster_cols[i]
        
        data[roll_item] = (data[roll_item] * (data['pitcher_pa'] + data['pa'] + data['bb']) - data[item]) / data['pitcher_pa']
    
    return data